<a href="https://colab.research.google.com/github/dave502/PyTorch/blob/main/HW_04.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

**Обучение классификатора картинок на примере CIFAR-100 (датасет можно изменить) сверточной сетью (самописной)**

In [1]:
import numpy as np
import torch

from torch import nn
from torch.nn import functional as F
from PIL import Image
from torchvision import transforms, datasets
from tqdm import tqdm
import warnings
warnings.filterwarnings("ignore")
from sklearn.model_selection import train_test_split

In [2]:
dataset = datasets.CIFAR10(root='data/', train=True, download=True)

X_train, X_valid = train_test_split(dataset, test_size=0.05, random_state=13)

  0%|          | 0/170498071 [00:00<?, ?it/s]

Extracting data/cifar-10-python.tar.gz to data/


Датасеты и лоадеры. Обучающий датасет с аугментацией

In [3]:
class ImgDataset(torch.utils.data.Dataset):
    
    def __init__(self, init_dataset, transform=None):
        self.__base_dataset = init_dataset
        self.transform = transform
        
    def __len__(self):
        return len(self.__base_dataset)
    
    def __getitem__(self, idx):
        img = self.__base_dataset[idx][0]
        if self.transform: img = self.transform(img)
        return img, self.__base_dataset[idx][1]
    
train_transforms = transforms.Compose([
    transforms.Resize(44),
    transforms.RandomCrop(32, padding=4),
    transforms.RandomHorizontalFlip(),
    transforms.RandomRotation(10),
    transforms.ToTensor(),
])
    
train_dataset = ImgDataset(X_train, train_transforms)
valid_dataset = ImgDataset(X_valid, transforms.ToTensor())

train_loader = torch.utils.data.DataLoader(train_dataset, batch_size=128,
                                          shuffle=True, num_workers=10)
valid_loader = torch.utils.data.DataLoader(valid_dataset, batch_size=128,
                                          num_workers=1)

Класс свёрточной сети (аналогичная рассмотренной на лекции)

In [34]:
class CNNet(nn.Module):
    
    def __init__(self):
        super(CNNet, self).__init__()
        self.l1_bn   = torch.nn.BatchNorm2d(3)
        self.l1_conv = torch.nn.Conv2d(3, 30, 3)
        self.l2_bn   = torch.nn.BatchNorm2d(30)
        self.l2_conv = torch.nn.Conv2d(30, 60, 3)
        self.l3_bn   = torch.nn.BatchNorm2d(60)
        self.l3_conv = torch.nn.Conv2d(60, 120, 3)
        self.l4_bn   = torch.nn.BatchNorm2d(120)
        self.l4_drop = nn.Dropout(0.2)
        
        self.fc_1 = torch.nn.Linear(480, 200)
        self.fc_drop = nn.Dropout(0.2)
        self.fc_2 = torch.nn.Linear(200, 60)
        self.fc_out = torch.nn.Linear(60, 10)
        
    def forward(self, x):
        
        x = self.l1_bn(x)
        x = self.l1_conv(x)
        x = F.relu(x)
        x = F.max_pool2d(x, 2)
        
        x = self.l2_bn(x)
        x = self.l2_conv(x)
        x = F.relu(x)
        x = F.max_pool2d(x, 2)
        
        x = self.l3_bn(x)
        x = self.l3_conv(x)
        x = F.relu(x)
        x = F.max_pool2d(x, 2)
        
        x = self.l4_bn(x)
        x = x.view(x.size(0), -1)
        x = self.l4_drop(x)
        
        x = self.fc_1(x)
        x = F.relu(x)
        x = self.fc_drop(x)
        x = self.fc_2(x)
        x = F.relu(x)
        
        return self.fc_out(x)

Инициализация объекта сети, оптимайзера и функции потерь

In [31]:
cnn = CNNet()
optimizer = torch.optim.Adam(cnn.parameters(), lr=0.01)
criterion = nn.CrossEntropyLoss()

Эпохи обучения с валидацией

In [35]:
for epoch in tqdm(range(10)):  
    cnn.train()
    for i, data in enumerate(train_loader, 0):
        
        inputs, labels = data[0], data[1]
        
        optimizer.zero_grad()
        outputs = cnn(inputs)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()
        
    cnn.eval()
    loss_accumed = 0
    for X, y in valid_loader:
        output = cnn(X)
        loss = criterion(output, y)
        loss_accumed += loss
    print("Epoch {} valid_loss {}".format(epoch, loss_accumed))

print('Training is finished!')

 10%|█         | 1/10 [01:09<10:24, 69.38s/it]

Epoch 0 valid_loss 32.66035842895508


 20%|██        | 2/10 [02:17<09:08, 68.60s/it]

Epoch 1 valid_loss 30.367944717407227


 30%|███       | 3/10 [03:26<08:00, 68.68s/it]

...

Epoch 7 valid_loss 24.21262550354004


 90%|█████████ | 9/10 [10:21<01:09, 69.19s/it]

Epoch 8 valid_loss 24.963573455810547


100%|██████████| 10/10 [11:30<00:00, 69.06s/it]

Epoch 9 valid_loss 24.96735382080078
Training is finished!


**Обучение классификатора картинок на примере CIFAR-100 (датасет можно изменить) через дообучение ImageNet Resnet-50**

Создание объекта предобученной сети resnet50. Инициализация датасета и лоадера без использования аугментации.

In [4]:
import torchvision.models as models
cnn_notr = models.resnet50(pretrained=True)

optimizer = torch.optim.Adam(cnn_notr.parameters(), lr=0.01)
criterion = nn.CrossEntropyLoss()

train_dataset_notransform = ImgDataset(X_train, transforms.ToTensor())
train_loader_notransform = torch.utils.data.DataLoader(train_dataset_notransform, batch_size=128,
                                          shuffle=True, num_workers=10)

Downloading: "https://download.pytorch.org/models/resnet50-0676ba61.pth" to /root/.cache/torch/hub/checkpoints/resnet50-0676ba61.pth


  0%|          | 0.00/97.8M [00:00<?, ?B/s]

Эпохи обучения с валидацией. Аугментация не применяется.

In [5]:
for epoch in tqdm(range(10)):  
    cnn_notr.train()
    for i, data in enumerate(train_loader_notransform, 0):
        
        inputs, labels = data[0], data[1]
        
        optimizer.zero_grad()
        outputs = cnn_notr(inputs)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()
        
    cnn_notr.eval()
    loss_accumed = 0
    for X, y in valid_loader:
        output = cnn_notr(X)
        loss = criterion(output, y)
        loss_accumed += loss
    print("Epoch {} valid_loss {}".format(epoch, loss_accumed))

print('Training is finished!')

 10%|█         | 1/10 [15:56<2:23:24, 956.02s/it]

Epoch 0 valid_loss 41608.640625


 20%|██        | 2/10 [31:36<2:06:15, 946.89s/it]

Epoch 1 valid_loss 41.17497253417969


 30%|███       | 3/10 [48:10<1:52:58, 968.39s/it]

...

Epoch 7 valid_loss 31.694194793701172


 90%|█████████ | 9/10 [2:27:47<16:31, 991.54s/it]

Epoch 8 valid_loss 18.81293487548828


100%|██████████| 10/10 [2:44:16<00:00, 985.65s/it]

Epoch 9 valid_loss 20.590185165405273
Training is finished!


*Визуально, полученные результаты метрики обученной Resnet-50 можно считать немного лучше, чем у самописной сети. Примущество Resnet можно объяснить более сложной архитектурой сети. То, что это предобученная сеть, возможно, тоже дало какой-то результат.*

**Обучение классификатора картинок на примере CIFAR-100 (датасет можно изменить) через дообучение ImageNet Resnet-50 с аугментацией (самописной, с использованием Pytorch встроенных методов)**

Создание объекта предобученной сети resnet50. Импорт ResNet50_Weights для явной инициализации сети почему-то не удался.

In [5]:
# from torchvision.models import resnet50, ResNet50_Weights
# cnn = resnet50(weights=ResNet50_Weights.DEFAULT)

import torchvision.models as models
cnn = models.resnet50(pretrained=True)

optimizer = torch.optim.Adam(cnn.parameters(), lr=0.01)
criterion = nn.CrossEntropyLoss()


Downloading: "https://download.pytorch.org/models/resnet50-0676ba61.pth" to /root/.cache/torch/hub/checkpoints/resnet50-0676ba61.pth


  0%|          | 0.00/97.8M [00:00<?, ?B/s]

Эпохи обучения с валидацией. Аугментация обучающего датасета применяется такая же, как и для самописной сети.

In [7]:
for epoch in tqdm(range(10)):  
    cnn.train()
    for i, data in enumerate(train_loader, 0):
        
        inputs, labels = data[0], data[1]
        
        optimizer.zero_grad()
        outputs = cnn(inputs)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()
        
    cnn.eval()
    loss_accumed = 0
    for X, y in valid_loader:
        output = cnn(X)
        loss = criterion(output, y)
        loss_accumed += loss
    print("Epoch {} valid_loss {}".format(epoch, loss_accumed))

print('Training is finished!')

 10%|█         | 1/10 [18:10<2:43:30, 1090.03s/it]

Epoch 0 valid_loss 44.117122650146484


 20%|██        | 2/10 [36:17<2:25:09, 1088.72s/it]

Epoch 1 valid_loss 34.90829849243164


 30%|███       | 3/10 [55:08<2:09:15, 1107.86s/it]

...

Epoch 7 valid_loss 30.609703063964844


 90%|█████████ | 9/10 [2:48:25<18:43, 1123.09s/it]

Epoch 8 valid_loss 29.744441986083984


100%|██████████| 10/10 [3:06:48<00:00, 1120.85s/it]

Epoch 9 valid_loss 39.91448974609375
Training is finished!


*Полученные результаты метрики Resnet-50 с обучением с аугментацией оказались хуже, чем без аугментации. Неожиданно.*